In [137]:
# Data manipulation
import pandas as pd
import numpy as np
# Evaluation of the model
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.size'] = 18
%matplotlib inline

MAX_EVALS = 20
import csv
from hyperopt import STATUS_OK
from timeit import default_timer as timer



DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [138]:
import json
import shlex
import subprocess
import logging
from pprint import pprint
import re
import os


In [139]:
def runthebenchmark(hyperparameters):
    os.chdir('/home/dsinghvi/project/progress/active/../')
    storeinfile(hyperparameters)
    out=subprocess.Popen(["python3","read_config_general.py","-n 8","-c400 400 800 4 4 4 1"], shell=False, stdout=subprocess.PIPE)
    logging.basicConfig(level=logging.DEBUG)
    output=out.stdout.read().decode('utf-8')
    print("output"+output)
    if len(output.split(" ")) > 5:
        value = output.split(" ")[5]
        value = float(value)
        print(value)
        return float(1/value)
    return 0;
    #logging.basicConfig(filename='app.log', filemode='a', format='%(name)s - %(levelname)s - %(message)s',level=logging.DEBUG)


In [140]:
def storeinfile(hyperparameters):
    data={"mpi": {"romio_ds_read": "enable", "romio_ds_write": "disable"}, "lfs": {"setstripe": {"size": "16777216", "count": 4}}}
    data["lfs"]["setstripe"]["size"] = int(hyperparameters["setstripe-size"])
    data["lfs"]["setstripe"]["count"] = int(hyperparameters["setstripe-count"])

    data["mpi"]["romio_ds_read"] = hyperparameters["romio_ds_read"]
    data["mpi"]["romio_ds_write"] = hyperparameters["romio_ds_write"]
    data["mpi"]["romio_cb_read"] = hyperparameters["romio_cb_read"]
    data["mpi"]["romio_cb_write"] = hyperparameters["romio_cb_write"]
    data["mpi"]["cb_buffer_size"] = str(int(hyperparameters["cb_buffer_size"]))
    with open("confex.json","w") as fp:
        json.dump(data,fp)
    print(data)


In [141]:
def objective(hyperparameters):
    global ITERATION
    ITERATION += 1
    start = timer()
    result=0
    while(result == 0):
        print("rerun")
        result = runthebenchmark(hyperparameters)
        print(result)
    run_time = timer() - start
    print({'loss': result, 'hyperparameters': hyperparameters, 'iteration': ITERATION, 'iteration_time': run_time, 'status': STATUS_OK})
    # Write to the csv file ('a' means append)
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([float(result), hyperparameters, ITERATION, run_time])
    return {'loss': float(result), 'hyperparameters': hyperparameters, 'iteration': ITERATION, 'iteration_time': run_time, 'status': STATUS_OK}


In [142]:
from hyperopt import hp
from hyperopt.pyll.stochastic import sample

space = {
    'romio_ds_read' : hp.choice('romio_ds_read',['enable']),
    'romio_ds_write' : hp.choice('romio_ds_write',['enable']),
    'romio_cb_read' : hp.choice('romio_cb_read',['enable']),
    'romio_cb_write' : hp.choice('romio_cb_write',['enable']),
    'cb_buffer_size' : 1048576*hp.quniform('cb_buffer_size',1,512,1),
    'setstripe-size' : 65536*(hp.qloguniform('setstripe-size',0,3,1)),
    'setstripe-count' : hp.quniform('setstripe-count',0,20,1)
    
}

In [143]:
x = sample(space)
params = x
print(x)
cb_buffer_size_dist = []
for _ in range(30):
    cb_buffer_size_dist.append(sample(space)['cb_buffer_size'])

{'cb_buffer_size': 397410304.0, 'romio_cb_read': 'enable', 'romio_cb_write': 'enable', 'romio_ds_read': 'enable', 'romio_ds_write': 'enable', 'setstripe-count': 1.0, 'setstripe-size': 196608.0}


In [144]:
from hyperopt import tpe

# Create the algorithm
tpe_algorithm = tpe.suggest


from hyperopt import Trials

# Record results
bayes_trials = Trials()

In [145]:
# File to save first results
print(os.getcwd())
out_file = '/home/dsinghvi/project/progress/active/result/gbm_trials.csv'
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)
writer.writerow(['loss', 'params', 'iteration', 'train_time'])
of_connection.close()

/home/dsinghvi/project/progress


In [ ]:
from hyperopt import fmin


ITERATION = 0

best = fmin(fn = objective, space = space, algo = tpe.suggest, trials = bayes_trials, max_evals = MAX_EVALS)

  0%|          | 0/20 [00:00<?, ?it/s, best loss: ?]

INFO:hyperopt.tpe:tpe_transform took 0.008683 seconds
INFO:hyperopt.tpe:TPE using 0 trials


rerun                                               
{'mpi': {'romio_ds_read': 'enable', 'romio_ds_write': 'enable', 'romio_cb_read': 'enable', 'romio_cb_write': 'enable', 'cb_buffer_size': '38797312'}, 'lfs': {'setstripe': {'size': 458752, 'count': 9}}}
outputS3D-IO 400-400-800-4-4-4-1 1125.01 15.26 13.89 543.22 45.78 86.29 0.17 0.14 458752 9 enable enable enable enable 38797312 

543.22                                              
0.0018408747836972127                               
{'loss': 0.0018408747836972127, 'hyperparameters': {'cb_buffer_size': 38797312.0, 'romio_cb_read': 'enable', 'romio_cb_write': 'enable', 'romio_ds_read': 'enable', 'romio_ds_write': 'enable', 'setstripe-count': 9.0, 'setstripe-size': 458752.0}, 'iteration': 1, 'iteration_time': 106.43382771220058, 'status': 'ok'}
  5%|▌         | 1/20 [01:46<33:42, 106.46s/it, best loss: 0.0018408747836972127]

INFO:hyperopt.tpe:tpe_transform took 0.010063 seconds
INFO:hyperopt.tpe:TPE using 1/1 trials with best loss 0.001841


rerun                                                                            
{'mpi': {'romio_ds_read': 'enable', 'romio_ds_write': 'enable', 'romio_cb_read': 'enable', 'romio_cb_write': 'enable', 'cb_buffer_size': '386924544'}, 'lfs': {'setstripe': {'size': 131072, 'count': 10}}}
outputS3D-IO 400-400-800-4-4-4-1 1095.57 15.26 14.26 1329.65 45.78 35.25 0.28 0.34 131072 10 enable enable enable enable 386924544 

1329.65                                                                          
0.0007520776144098071                                                            
{'loss': 0.0007520776144098071, 'hyperparameters': {'cb_buffer_size': 386924544.0, 'romio_cb_read': 'enable', 'romio_cb_write': 'enable', 'romio_ds_read': 'enable', 'romio_ds_write': 'enable', 'setstripe-count': 10.0, 'setstripe-size': 131072.0}, 'iteration': 2, 'iteration_time': 55.68092682491988, 'status': 'ok'}
 10%|█         | 2/20 [02:42<27:22, 91.24s/it, best loss: 0.0007520776144098071] 

...

rerun                                                                           
{'mpi': {'romio_ds_read': 'enable', 'romio_ds_write': 'enable', 'romio_cb_read': 'enable', 'romio_cb_write': 'enable', 'cb_buffer_size': '192937984'}, 'lfs': {'setstripe': {'size': 655360, 'count': 0}}}
outputS3D-IO 400-400-800-4-4-4-1 1061.96 15.26 14.71 550.05 45.78 85.22 0.14 0.36 655360 0 enable enable enable enable 192937984 

550.05                                                                          
0.0018180165439505502                                                           
{'loss': 0.0018180165439505502, 'hyperparameters': {'cb_buffer_size': 192937984.0, 'romio_cb_read': 'enable', 'romio_cb_write': 'enable', 'romio_ds_read': 'enable', 'romio_ds_write': 'enable', 'setstripe-count': 0.0, 'setstripe-size': 655360.0}, 'iteration': 15, 'iteration_time': 105.80711520649493, 'status': 'ok'}
 75%|███████▌  | 15/20 [19:38<06:49, 81.99s/it, best loss: 0.000722548573327842]

INFO:hyperopt.tpe:tpe_transform took 0.009408 seconds
INFO:hyperopt.tpe:TPE using 15/15 trials with best loss 0.000723


rerun                                                                           
{'mpi': {'romio_ds_read': 'enable', 'romio_ds_write': 'enable', 'romio_cb_read': 'enable', 'romio_cb_write': 'enable', 'cb_buffer_size': '312475648'}, 'lfs': {'setstripe': {'size': 131072, 'count': 9}}}
outputS3D-IO 400-400-800-4-4-4-1 820.82 15.26 19.04 1192.24 45.78 39.32 0.09 0.56 131072 9 enable enable enable enable 312475648 

1192.24                                                                         
0.0008387572971884855                                                           
{'loss': 0.0008387572971884855, 'hyperparameters': {'cb_buffer_size': 312475648.0, 'romio_cb_read': 'enable', 'romio_cb_write': 'enable', 'romio_ds_read': 'enable', 'romio_ds_write': 'enable', 'setstripe-count': 9.0, 'setstripe-size': 131072.0}, 'iteration': 16, 'iteration_time': 64.20483140088618, 'status': 'ok'}
 80%|████████  | 16/20 [20:42<05:06, 76.66s/it, best loss: 0.000722548573327842]

INFO:hyperopt.tpe:tpe_transform took 0.009522 seconds
INFO:hyperopt.tpe:TPE using 16/16 trials with best loss 0.000723


rerun                                                                           
{'mpi': {'romio_ds_read': 'enable', 'romio_ds_write': 'enable', 'romio_cb_read': 'enable', 'romio_cb_write': 'enable', 'cb_buffer_size': '381681664'}, 'lfs': {'setstripe': {'size': 851968, 'count': 16}}}
 80%|████████  | 16/20 [20:42<05:06, 76.66s/it, best loss: 0.000722548573327842]

In [ ]:
print(best)
d=  best
#print(bayes_trials.results)
bayes_trials_results = sorted(bayes_trials.results, key = lambda x: x['loss'])
#bayes_trials_results[:1]

In [ ]:
results = pd.read_csv(out_file)

# Sort with best scores on top and reset index for slicing
results.sort_values('train_time', ascending = True, inplace = True)
results.reset_index(inplace = True, drop = True)
results.head()


In [ ]:
import ast

# Convert from a string to a dictionary
ast.literal_eval(results.loc[0, 'params'])
best_bayes_params = ast.literal_eval(results.loc[0, 'params']).copy()
print(best_bayes_params)

In [ ]:
bayes_params = pd.DataFrame(columns = list(ast.literal_eval(results.loc[0, 'params']).keys()),
                            index = list(range(len(results))))

# Add the results with each parameter a different column
for i, params in enumerate(results['params']):
    bayes_params.loc[i, :] = list(ast.literal_eval(params).values())
bayes_params['train_time'] = results['train_time']    
bayes_params['loss'] = results['loss']
bayes_params['iteration'] = results['iteration']

bayes_params.head()

In [ ]:
plt.figure(figsize = (20, 8))
plt.rcParams['font.size'] = 18

# Density plots of the learning rate distributions 
sns.kdeplot(bayes_params['loss'], label = 'Loss Variation', linewidth = 2)
plt.legend()
plt.xlabel('Loss'); plt.ylabel('Density'); plt.title('Loss Distribution');


In [ ]:
plt.figure(figsize = (20, 8))
plt.rcParams['font.size'] = 18
cb_buffer_size_dist_now = []
for _ in range(30):
    cb_buffer_size_dist_now.append(sample(space)['cb_buffer_size'])
# Density plots of the learning rate distributions 
sns.kdeplot(bayes_params['cb_buffer_size'], label = 'cb_buffer_size', linewidth = 2)
sns.kdeplot(cb_buffer_size_dist,color="red",linewidth=2)
sns.kdeplot(cb_buffer_size_dist_now,color="green",linewidth=2)
plt.legend()
plt.xlabel('cb_buffer_size'); plt.ylabel('Density'); plt.title('cb_buffer_size Distribution');